# Repeated Measures ANOVA and Hierarchical Models

## Question 1:

In this exercise, we will investigate how reaction times vary across three experimental conditions: **Baseline**, **Task**, and **Post-Task**. The dataset includes reaction times (in milliseconds) measured for 12 participants under each condition. The goal of the exercise is to determine if there are significant differences in reaction times across these three conditions.

Our approach to analyzing the data is structured as follows:

1. **Repeated Measures ANOVA:** We will conduct a Repeated Measures ANOVA to examine whether there are significant differences in reaction times between the conditions. We will calculate the F-statistic, degrees of freedom, and the p-value to evaluate this.

2. **Sphericity Assumption:** using Mauchly's test of sphericity. If the assumption is violated, we will apply the Greenhouse-Geisser correction to adjust the degrees of freedom and recalculate the p-value.

By completing this exercise, we will gain deeper understanding of how to conduct a repeated measurements ANOVA, interpret the results, and evaluate the p-value to make an informed decision about whether to reject the null hypothesis. The interpretation will focus on understanding how reaction times change from Baseline to Task and Post-Task, shedding light on the effects of the experimental conditions.

## Q1 Solution:

In [72]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pingouin as pg
import statsmodels.api as sm
from statsmodels.formula.api import mixedlm

We are using pandas to create the dataset based on the problem statement.

In [2]:
# Create a dataframe for reaction times under three conditions
data = {
    'Participant': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'Baseline': [250, 260, 270, 240, 265, 275, 255, 245, 270, 260, 250, 265],
    'Task': [240, 245, 255, 235, 250, 260, 245, 235, 260, 250, 240, 255],
    'Post-Task': [230, 235, 245, 225, 240, 250, 235, 225, 250, 240, 230, 245]
}

df = pd.DataFrame(data)

In [3]:
df

,Participant,Baseline,Task,Post-Task
0,1,250,240,230
1,2,260,245,235
2,3,270,255,245
3,4,240,235,225
4,5,265,250,240
5,6,275,260,250
6,7,255,245,235
7,8,245,235,225
8,9,270,260,250
9,10,260,250,240


## Convert the dataframe to a long format for Pingouin

In [4]:
df_long = df.melt(id_vars=['Participant'],
                  value_vars=['Baseline', 'Task', 'Post-Task'],
                  var_name='Condition',
                  value_name='Reaction_Time')


In [34]:
df_long

,Participant,Condition,Reaction_Time
0,1,Baseline,250
1,2,Baseline,260
2,3,Baseline,270
3,4,Baseline,240
4,5,Baseline,265
5,6,Baseline,275
6,7,Baseline,255
7,8,Baseline,245
8,9,Baseline,270
9,10,Baseline,260


## Perform Repeated Measures ANOVA

In [5]:
anova_results = pg.rm_anova(data=df_long,
                            dv='Reaction_Time',  # Dependent variable
                            within='Condition',  # Within-subject variable
                            subject='Participant',  # Subject identifier
                            detailed=True)

Display the ANOVA table

In [6]:
# Display the ANOVA table
print("ANOVA Table:")
print(anova_results)

ANOVA Table:
      Source           SS  DF           MS           F         p-unc  \
0  Condition  2712.500000   2  1356.250000  421.235294  2.899994e-18   
1      Error    70.833333  22     3.219697         NaN           NaN   

        ng2  eps  
0  0.470206  0.5  
1       NaN  NaN  


## Test sphericity assumption

In [35]:
sphericity_results = pg.sphericity(data=df_long,
                                   dv='Reaction_Time',
                                   within='Condition',
                                   subject='Participant')

# Extract relevant details from the SpherResults object
is_sphericity_assumed = sphericity_results.spher
w_stat = sphericity_results.W
chi2_stat = sphericity_results.chi2
dof = sphericity_results.dof
p_val = sphericity_results.pval

# Check if sphericity is violated and apply corrections if necessary
if not is_sphericity_assumed:
    print("Sphericity is violated. Applying Greenhouse-Geisser correction.")
    anova_results_corrected = pg.rm_anova(data=df_long,
                                          dv='Reaction_Time',
                                          within='Condition',
                                          subject='Participant',
                                          detailed=True,
                                          correction=True)
else:
    print("Sphericity is not violated. No correction needed.")
    anova_results_corrected = anova_results

Sphericity is not violated. No correction needed.


Display the ANOVA and sphericity results

In [37]:
# Step 5: Display results
print("\nRepeated Measures ANOVA Results:\n", anova_results_corrected)
print("\nSphericity Test Results:\n")
print(f"Sphericity Assumed: {is_sphericity_assumed}")
print(f"W statistic: {w_stat:.4f}, Chi-Square: {chi2_stat:.4f}, df: {dof}, p-value: {p_val:.4f}")




Repeated Measures ANOVA Results:
       Source           SS  DF           MS           F         p-unc  \
0  Condition  2712.500000   2  1356.250000  421.235294  2.899994e-18   
1      Error    70.833333  22     3.219697         NaN           NaN   

        ng2  eps  
0  0.470206  0.5  
1       NaN  NaN  

Sphericity Test Results:

Sphericity Assumed: True
W statistic: 0.6212, Chi-Square: 4.7614, df: 2, p-value: 0.0925


## Interpret the results

In [40]:
# Interpretation
if anova_results_corrected['p-unc'][0] < 0.01:  # 1% significance level
    print("\nThe p-value is below 0.01. We reject the null hypothesis.")
    print("There is a statistically significant difference in reaction times across the conditions.")
else:
    print("\nThe p-value is above 0.01. We fail to reject the null hypothesis.")
    print("There is no statistically significant difference in reaction times across the conditions.")


The p-value is below 0.01. We reject the null hypothesis.
There is a statistically significant difference in reaction times across the conditions.


# Question 2:

In this exercise, we will investigate the reaction times for three different tasks (A, B, and C) measured across 15 participants, each completing 5 trials per task. The goal is to determine if there are significant differences in reaction times between the tasks while accounting for variability between participants.

Our approach to analyzing the data is structured as follows:

1. **Hierarchical Model:** We will estimate the fixed effects to assess how reaction times differ between the tasks and the random effects to account for individual differences in baseline reaction times and variability across trials.

2. ** Repeated Measures ANOVA :** to test for significant differences in reaction times across the three tasks. This will allow us to compare the performance across tasks while adjusting for within-subject correlation.


By completing this exercise, we will interpret the results, focusing on whether there are statistically significant differences in reaction times between the tasks. We will examine the fixed effects for task differences and the random effects for participant-specific variability, providing insights into how each task impacts reaction time across participants.

## Q2 Solution:

In [63]:
# Data for the table
data = {
    'Participant': [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8,
                    9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15],
    'Task': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C',
             'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C',
             'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
    'Trial 1': [250, 260, 270, 240, 250, 260, 255, 265, 275, 245, 255, 265, 250, 260, 270, 240, 250, 260,
                255, 265, 275, 245, 255, 265, 250, 260, 270, 240, 250, 260, 255, 264, 275, 245, 255, 265,
                250, 260, 270, 240, 250, 260, 245, 255, 265],
    'Trial 2': [245, 255, 265, 235, 245, 255, 250, 260, 270, 240, 250, 260, 245, 255, 265, 235, 245, 255,
                250, 260, 270, 240, 250, 260, 245, 255, 265, 235, 245, 255, 250, 260, 270, 240, 250, 260,
                245, 255, 265, 235, 245, 255, 240, 250, 260],
    'Trial 3': [240, 250, 260, 230, 240, 250, 245, 255, 265, 235, 245, 255, 240, 250, 260, 230, 240, 250,
                245, 255, 265, 235, 245, 255, 240, 250, 260, 230, 240, 250, 245, 255, 265, 235, 245, 255,
                240, 250, 260, 230, 240, 250, 235, 245, 255],
    'Trial 4': [255, 265, 275, 245, 255, 265, 250, 260, 270, 255, 265, 275, 255, 265, 275, 245, 255, 265,
                260, 270, 280, 250, 260, 270, 255, 265, 275, 245, 255, 265, 260, 270, 280, 250, 260, 270,
                255, 265, 275, 245, 255, 265, 250, 260, 270],
    'Trial 5': [250, 260, 270, 240, 250, 260, 255, 265, 275, 245, 255, 265, 250, 260, 270, 240, 250, 260,
                255, 265, 275, 245, 255, 265, 250, 260, 270, 240, 250, 260, 255, 260, 275, 245, 255, 265,
                250, 260, 270, 240, 250, 260, 245, 255, 265]
}

# Create a DataFrame
df = pd.DataFrame(data)

    Participant Task  Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
0             1    A      250      245      240      255      250
1             1    B      260      255      250      265      260
2             1    C      270      265      260      275      270
3             2    A      240      235      230      245      240
4             2    B      250      245      240      255      250
5             2    C      260      255      250      265      260
6             3    A      255      250      245      250      255
7             3    B      265      260      255      260      265
8             3    C      275      270      265      270      275
9             4    A      245      240      235      255      245
10            4    B      255      250      245      265      255
11            4    C      265      260      255      275      265
12            5    A      250      245      240      255      250
13            5    B      260      255      250      265      260
14        

In [64]:
# Display the DataFrame
print(df)

    Participant Task  Trial 1  Trial 2  Trial 3  Trial 4  Trial 5
0             1    A      250      245      240      255      250
1             1    B      260      255      250      265      260
2             1    C      270      265      260      275      270
3             2    A      240      235      230      245      240
4             2    B      250      245      240      255      250
5             2    C      260      255      250      265      260
6             3    A      255      250      245      250      255
7             3    B      265      260      255      260      265
8             3    C      275      270      265      270      275
9             4    A      245      240      235      255      245
10            4    B      255      250      245      265      255
11            4    C      265      260      255      275      265
12            5    A      250      245      240      255      250
13            5    B      260      255      250      265      260
14        

In [65]:
# Reshape the DataFrame to long format
df_long = pd.melt(df, id_vars=['Participant', 'Task'], var_name='Trial', value_name='Score')

# Display the DataFrame in long format
print(df_long)

     Participant Task    Trial  Score
0              1    A  Trial 1    250
1              1    B  Trial 1    260
2              1    C  Trial 1    270
3              2    A  Trial 1    240
4              2    B  Trial 1    250
..           ...  ...      ...    ...
220           14    B  Trial 5    250
221           14    C  Trial 5    260
222           15    A  Trial 5    245
223           15    B  Trial 5    255
224           15    C  Trial 5    265

[225 rows x 4 columns]


## Perform Mixed-Effects Model

In [73]:
# Fit the Mixed-Effects Model using statsmodels
model = mixedlm("Score ~ Task", df_long, groups=df_long["Participant"])
mixed_model = model.fit()

Display the ANOVA table

In [74]:
# Print Results
print("Mixed-Effects Model results:")
print(mixed_model.summary())


Mixed-Effects Model results:
         Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: Score    
No. Observations:  225     Method:             REML     
No. Groups:        15      Scale:              28.5541  
Min. group size:   15      Log-Likelihood:     -712.5374
Max. group size:   15      Converged:          Yes      
Mean group size:   15.0                                 
--------------------------------------------------------
           Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
--------------------------------------------------------
Intercept 244.933    1.474 166.225 0.000 242.045 247.821
Task[T.B]   9.920    0.873  11.368 0.000   8.210  11.630
Task[T.C]  20.000    0.873  22.920 0.000  18.290  21.710
Group Var  26.857    2.102                              



## Perform repeated measures ANOVA

In [77]:
# Perform repeated measures ANOVA using pingouin
anova_results = pg.rm_anova(dv='Score', within='Task', subject='Participant', data=df_long)


Display the ANOVA table

In [78]:
# Display the ANOVA table
print("\nRepeated Measures ANOVA results:")
print(anova_results)


Repeated Measures ANOVA results:
  Source  ddof1  ddof2        F         p-unc       ng2  eps
0   Task      2     28  46876.0  4.473725e-50  0.712799  0.5


## Interpret the results

In [92]:
# Mixed-Effects Model Interpretation
print("\n--- Mixed-Effects Model Interpretation ---")

# Get p-values for the fixed effects (Task[T.B] and Task[T.C])
task_b_pvalue = mixed_model.pvalues["Task[T.B]"]
task_c_pvalue = mixed_model.pvalues["Task[T.C]"]

print("The fixed effects for Task[T.B] and Task[T.C] show how much each Task (B and C) differs from the baseline (Task A).")

# Interpretation for Task B
if task_b_pvalue < 0.05:
    print(f"Task B has a significant effect on Score compared to Task A (p = {task_b_pvalue:.4f}).")
else:
    print(f"Task B does not have a significant effect on Score compared to Task A (p = {task_b_pvalue:.4f}).")

# Interpretation for Task C
if task_c_pvalue < 0.05:
    print(f"Task C has a significant effect on Score compared to Task A (p = {task_c_pvalue}).")
else:
    print(f"Task C does not have a significant effect on Score compared to Task A (p = {task_c_pvalue}).")

# Repeated Measures ANOVA Interpretation
print("\n--- Repeated Measures ANOVA Interpretation ---")
print("The F-value indicates the ratio of variance between tasks to the variance within participants.")
print("A significant p-value (usually < 0.05) means that there are differences in scores across tasks.")
print("The np2 (partial eta-squared) indicates the proportion of variance explained by Task.")

# Example interpretation for Repeated Measures ANOVA
anova_pvalue = anova_results['p-unc'][0]  # Corrected column name
if anova_pvalue < 0.05:
    print(f"The p-value for Task in the ANOVA is {anova_pvalue}, which is significant.")
    print(f"This suggests that there are significant differences in scores between the different Tasks.")
else:
    print(f"The p-value for Task in the ANOVA is {anova_pvalue}, which is not significant.")
    print("This suggests that Task does not significantly affect the scores.")

print("\nEffect Size Interpretation:")
print(f"The np2 (effect size) from the ANOVA is {anova_results['ng2'][0]}. This indicates how much of the total variance in scores is explained by the Task factor.")



--- Mixed-Effects Model Interpretation ---
The fixed effects for Task[T.B] and Task[T.C] show how much each Task (B and C) differs from the baseline (Task A).
Task B has a significant effect on Score compared to Task A (p = 0.0000).
Task C has a significant effect on Score compared to Task A (p = 2.947685299767912e-116).

--- Repeated Measures ANOVA Interpretation ---
The F-value indicates the ratio of variance between tasks to the variance within participants.
A significant p-value (usually < 0.05) means that there are differences in scores across tasks.
The np2 (partial eta-squared) indicates the proportion of variance explained by Task.
The p-value for Task in the ANOVA is 4.473724553748408e-50, which is significant.
This suggests that there are significant differences in scores between the different Tasks.

Effect Size Interpretation:
The np2 (effect size) from the ANOVA is 0.7127993218096733. This indicates how much of the total variance in scores is explained by the Task factor.